<a href="https://colab.research.google.com/github/karthik111/video_anomaly_detection/blob/master/notebooks/VIT_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install decord

In [2]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

url_c = 'https://farm4.staticflickr.com/3545/3409800178_24c6f790e6_z.jpg'
image_c = Image.open(requests.get(url_c, stream=True).raw)

url_d = 'https://farm6.staticflickr.com/5332/9374828651_07f9433075_z.jpg'
image_d = Image.open(requests.get(url_d, stream=True).raw)

url_d1 = 'https://farm3.staticflickr.com/2556/4228514131_81f3416db3_z.jpg'
image_d1 = Image.open(requests.get(url_d1, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

#image = [np.random.randn(3, 224, 224) for _ in range(10)]

inputs = processor(images=[image, image_c, image_d, image_d1], return_tensors="pt")

outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [3]:
inputs['pixel_values'].shape

torch.Size([4, 3, 224, 224])

In [4]:
last_hidden_states.shape

torch.Size([4, 197, 768])

In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
video_path = ['/content/drive/MyDrive/colab/data/kart.mov',
              '/content/drive/MyDrive/colab/data/Assault007_x264.mp4',
              '/content/drive/MyDrive/colab/data/Explosion002_x264.mp4']


In [7]:
import decord
vr = decord.VideoReader(video_path[2])

In [8]:
vr.get_avg_fps()

30.0

In [9]:
# Get the total number of frames in the video
num_frames = len(vr)

# Read all frames
frames = [vr[i].asnumpy() for i in range(num_frames)]

# Display some information
print(f"Total number of frames: {num_frames}")
print(f"Shape of a single frame: {frames[0].shape}")

Total number of frames: 4013
Shape of a single frame: (240, 320, 3)


In [10]:
inputs = processor(images=frames[:32], return_tensors="pt")

In [11]:
inputs['pixel_values'].shape

torch.Size([32, 3, 224, 224])

In [12]:
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [13]:
last_hidden_states.shape

torch.Size([32, 197, 768])

In [14]:
# prompt: flatten above [32,197,168] vector to [32,192*168] vector

flattened_vector = last_hidden_states.reshape(last_hidden_states.shape[0], -1)
flattened_vector.shape


torch.Size([32, 151296])

In [ ]:
# prompt: apply convolution to convert a [32,151296]dimension feature tensor to a [32,2048] feature tensor

import torch.nn as nn
conv = nn.Conv1d(in_channels=151296, out_channels=2048, kernel_size=768)
flattened_vector = conv(flattened_vector)
flattened_vector.shape


In [1]:
import torch
import torch.nn as nn

# Define input tensor with shape [32, 151296]
input_tensor = torch.randn(32, 151296)

# Define a 1D convolutional layer
# We need to set the number of input channels to 1, as we have a single feature sequence per sample
# Output channels should be set to 2048 to match the desired output dimension
# The kernel size and stride need to be determined to match the output size requirement
class Conv1DLayer(nn.Module):
    def __init__(self):
        super(Conv1DLayer, self).__init__()
        # Define the convolutional layer
        # in_channels = 1 (since we're treating each sequence as a single channel input)
        # out_channels = 2048 (desired output features)
        # kernel_size = (appropriate value, we'll calculate)
        # stride = (appropriate value, we'll calculate)
        # We'll use some padding to help adjust the size

        self.conv1d = nn.Conv1d(in_channels=1, out_channels=2048, kernel_size=73, stride=73)

    def forward(self, x):
        # Add an extra dimension to match the input shape expected by Conv1d: (batch_size, in_channels, length)
        x = x.unsqueeze(1)
        x = self.conv1d(x)
        # Remove the extra dimension added earlier
        x = x.squeeze(2)
        return x

# Create the model
model = Conv1DLayer()

# Apply the model to the input tensor
output_tensor = model(input_tensor)

print("Output tensor shape:", output_tensor.shape)


Output tensor shape: torch.Size([32, 2048, 2072])


In [3]:
import torch
import torch.nn as nn

# Define input tensor with shape [32, 151296]
input_tensor = torch.randn(32, 151296)

# Define a 1D convolutional layer
class Conv1DLayer(nn.Module):
    def __init__(self):
        super(Conv1DLayer, self).__init__()
        # Define the convolutional layer
        # in_channels = 1 (since we're treating each sequence as a single channel input)
        # out_channels = 2048 (desired output features)
        # kernel_size = appropriate value to achieve desired output
        # stride = appropriate value to achieve desired output
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=2048, kernel_size=73, stride=73)

    def forward(self, x):
        # Add an extra dimension to match the input shape expected by Conv1d: (batch_size, in_channels, length)
        x = x.unsqueeze(1)
        x = self.conv1d(x)
        # Remove the extra dimension added earlier
        x = x.squeeze(2)
        return x

# Create the model
model = Conv1DLayer()

# Apply the model to the input tensor
output_tensor = model(input_tensor)

print("Output tensor shape:", output_tensor.shape)



Output tensor shape: torch.Size([32, 2048, 2072])


In [11]:
import torch
import torch.nn as nn

# Define input tensor with shape [32, 151296]
input_tensor = torch.randn(32, 151296)

# Define an average pooling layer
class AveragePoolingLayer(nn.Module):
    def __init__(self):
        super(AveragePoolingLayer, self).__init__()
        # Define the average pooling layer
        # kernel_size and stride should be chosen to reduce the input dimension to the desired output dimension
        self.avg_pool = nn.AvgPool1d(kernel_size=74, stride=74)

    def forward(self, x):
        # Add an extra dimension to match the input shape expected by AvgPool1d: (batch_size, channels, length)
        x = x.unsqueeze(1)
        x = self.avg_pool(x)
        # Remove the extra dimension added earlier
        x = x.squeeze(1)
        return x

# Create the model
model = AveragePoolingLayer()

# Apply the model to the input tensor
output_tensor = model(input_tensor)

print("Output tensor shape:", output_tensor.shape)


Output tensor shape: torch.Size([32, 2044])


In [12]:
151296/2048

73.875